In [278]:
from bs4 import BeautifulSoup
import requests
import re

link = 'https://coinmarketcap.com/pt-br/'
req = requests.get(link)
site = BeautifulSoup(req.text, 'html.parser')

#### Extraindo o nome das 10 primeiras moedas na tabela

In [279]:
tabela = site.find('tbody')
moedas_classe = tabela.find_all(class_='ehyBa-d')
moedas_nome = [moeda.contents[0] for moeda in moedas_classe]
moedas_nome

['Bitcoin',
 'Ethereum',
 'Tether',
 'BNB',
 'Solana',
 'USDC',
 'XRP',
 'Dogecoin',
 'Toncoin',
 'Cardano']

#### Extraindo o preço das moedas

In [280]:
preco_classe = tabela.find_all(class_='gPTgRa')
precos = [preco.contents[0].text for preco in preco_classe]
precos

['R$356,234.27',
 'R$19,269.49',
 'R$5.35',
 'R$3,264.01',
 'R$798.49',
 'R$5.36',
 'R$2.62',
 'R$0.728',
 'R$43.21',
 'R$2.23']

#### Extraindo a informação de variação das moedas

Aqui as variações estão distribuídas da seguinte forma: O primeiro item é a variação de 1h, o segundo é a variação de 24h e o terceiro é a variação de 7 dias. Os três primeiros itens da lista são as variações da primeria moeda da tabela, os próximos 3 são as variaçoes da segunda moeda e assim vai.

In [281]:
variacoes = tabela.find_all(string=re.compile("%"))
variacoes

['0.13%',
 '0.30%',
 '4.06%',
 '0.15%',
 '0.77%',
 '2.23%',
 '0.17%',
 '0.37%',
 '0.53%',
 '0.19%',
 '0.30%',
 '8.86%',
 '0.54%',
 '2.79%',
 '7.32%',
 '0.17%',
 '0.34%',
 '0.61%',
 '0.31%',
 '0.68%',
 '1.11%',
 '0.00%',
 '0.54%',
 '7.00%',
 '0.11%',
 '1.96%',
 '14.47%',
 '0.10%',
 '0.54%',
 '5.98%']

#### Extraindo a informação de capitalização de mercado para cada moeda

A capitalização de mercado é:<br> O valor total de mercado do fornecimento em circulação de uma criptomoeda. É semelhante à capitalização do free float no mercado de ações.

Capitalizado de Mercado = Preço Atual x Fornecimento em Circulação.

In [282]:
cap_elementos = tabela.find_all(class_='hwOFkt')
cap_mercado = [cap.contents[0] for cap in cap_elementos]
cap_mercado

['R$7,022,237,295,920',
 'R$2,315,249,946,493',
 'R$602,734,763,544',
 'R$481,737,186,831',
 'R$368,747,220,300',
 'R$174,121,066,335',
 'R$145,755,144,739',
 'R$105,375,955,266',
 'R$105,136,085,404',
 'R$79,581,823,099']

#### Extraindo o volume de cada moeda

O volume é uma medida de quanto de uma criptomoeda foi negociada nas últimas 24 horas.

In [283]:
volume_elementos = tabela.find_all(class_='bbHOdE')
volume_24h = [volume.contents[0].text for volume in volume_elementos]
volume_24h

['R$7.02T',
 'R$65,264,938,868',
 'R$2.32T',
 'R$48,289,381,483',
 'R$602.73B',
 'R$161,287,835,245',
 'R$481.74B',
 'R$7,545,040,684',
 'R$368.75B',
 'R$5,482,982,773',
 'R$174.12B',
 'R$12,891,947,387',
 'R$145.76B',
 'R$3,787,312,738',
 'R$105.38B',
 'R$1,959,632,334',
 'R$105.14B',
 'R$1,342,147,322',
 'R$79.58B',
 'R$895,347,686']

#### Existem outros elementos no site com a mesma classe que o volume, na seguinte célula eu trato esse problema, garantindo que estou extraindo apenas o volume.

In [284]:
for volume in volume_24h:
  if volume.endswith('T') or volume.endswith('B'):
    volume_24h.remove(volume)
volume_24h

['R$65,264,938,868',
 'R$48,289,381,483',
 'R$161,287,835,245',
 'R$7,545,040,684',
 'R$5,482,982,773',
 'R$12,891,947,387',
 'R$3,787,312,738',
 'R$1,959,632,334',
 'R$1,342,147,322',
 'R$895,347,686']

#### Extraindo o fornecimento circulante de cada moeda

O fornecimento circulante é a quantidade de moedas que estão circulando no mercado e estão nas mãos do público. É parecido com a flutuação de ações no mercado de ações.

In [285]:
fornecimento_elementos = tabela.find_all(class_='hhmVNu')
fornecimento_circulante = [each.text for each in fornecimento_elementos]
fornecimento_circulante

['19,713,231 BTC',
 '120,154,584 ETH',
 '112,559,949,271 USDT',
 '147,584,076 BNB',
 '461,808,053 SOL',
 '32,490,144,305 USDC',
 '55,570,463,728 XRP',
 '144,737,596,384 DOGE',
 '2,432,953,027 TON',
 '35,728,563,624 ADA']

## Criando um dicionário com as informações de cada moeda

In [286]:
moedas = {moeda: {"Preço": precos[i]} for i, moeda in enumerate(moedas_nome)}

for i, moeda in enumerate(moedas):
  moedas[moeda]['Variação 1h'] = variacoes.pop(i-i)
  moedas[moeda]['Variação 24h'] = variacoes.pop(i-i)
  moedas[moeda]['Variação 7d'] = variacoes.pop(i-i)

for i, moeda in enumerate(moedas):
  moedas[moeda]['Cap. de Mercado'] = cap_mercado[i]

for i, moeda in enumerate(moedas):
  moedas[moeda]['Volume (24h)'] = volume_24h[i]

for i, moeda in enumerate(moedas):
  moedas[moeda]['Fornecimento Circulante'] = fornecimento_circulante[i]

for moeda in moedas:
  print(moeda, moedas[moeda])
  print()

Bitcoin {'Preço': 'R$356,234.27', 'Variação 1h': '0.13%', 'Variação 24h': '0.30%', 'Variação 7d': '4.06%', 'Cap. de Mercado': 'R$7,022,237,295,920', 'Volume (24h)': 'R$65,264,938,868', 'Fornecimento Circulante': '19,713,231 BTC'}

Ethereum {'Preço': 'R$19,269.49', 'Variação 1h': '0.15%', 'Variação 24h': '0.77%', 'Variação 7d': '2.23%', 'Cap. de Mercado': 'R$2,315,249,946,493', 'Volume (24h)': 'R$48,289,381,483', 'Fornecimento Circulante': '120,154,584 ETH'}

Tether {'Preço': 'R$5.35', 'Variação 1h': '0.17%', 'Variação 24h': '0.37%', 'Variação 7d': '0.53%', 'Cap. de Mercado': 'R$602,734,763,544', 'Volume (24h)': 'R$161,287,835,245', 'Fornecimento Circulante': '112,559,949,271 USDT'}

BNB {'Preço': 'R$3,264.01', 'Variação 1h': '0.19%', 'Variação 24h': '0.30%', 'Variação 7d': '8.86%', 'Cap. de Mercado': 'R$481,737,186,831', 'Volume (24h)': 'R$7,545,040,684', 'Fornecimento Circulante': '147,584,076 BNB'}

Solana {'Preço': 'R$798.49', 'Variação 1h': '0.54%', 'Variação 24h': '2.79%', 'Variaç

## Criando o dataframe

In [287]:
import pandas as pd

moedas_df = pd.DataFrame(moedas)
moedas_df

,Bitcoin,Ethereum,Tether,BNB,Solana,USDC,XRP,Dogecoin,Toncoin,Cardano
Preço,"R$356,234.27","R$19,269.49",R$5.35,"R$3,264.01",R$798.49,R$5.36,R$2.62,R$0.728,R$43.21,R$2.23
Variação 1h,0.13%,0.15%,0.17%,0.19%,0.54%,0.17%,0.31%,0.00%,0.11%,0.10%
Variação 24h,0.30%,0.77%,0.37%,0.30%,2.79%,0.34%,0.68%,0.54%,1.96%,0.54%
Variação 7d,4.06%,2.23%,0.53%,8.86%,7.32%,0.61%,1.11%,7.00%,14.47%,5.98%
Cap. de Mercado,"R$7,022,237,295,920","R$2,315,249,946,493","R$602,734,763,544","R$481,737,186,831","R$368,747,220,300","R$174,121,066,335","R$145,755,144,739","R$105,375,955,266","R$105,136,085,404","R$79,581,823,099"
Volume (24h),"R$65,264,938,868","R$48,289,381,483","R$161,287,835,245","R$7,545,040,684","R$5,482,982,773","R$12,891,947,387","R$3,787,312,738","R$1,959,632,334","R$1,342,147,322","R$895,347,686"
Fornecimento Circulante,"19,713,231 BTC","120,154,584 ETH","112,559,949,271 USDT","147,584,076 BNB","461,808,053 SOL","32,490,144,305 USDC","55,570,463,728 XRP","144,737,596,384 DOGE","2,432,953,027 TON","35,728,563,624 ADA"


### Criando uma função para consultar uma moeda específica

In [288]:
def consultar_moeda():
  moeda = input("Qual moeda você deseja consultar?")
  moeda = moeda.casefold()
  moeda = moeda.title()
  if moeda == 'Bnb':
    moeda = moeda.upper()
  elif moeda == 'Usdc':
    moeda = moeda.upper()
  elif moeda == 'Xrp':
    moeda = moeda.upper()
  def consultar(moeda):
    try:
      print(moeda)
      return moedas[moeda]
    except:
      print("Moeda inválida!")
  return consultar(moeda)

consultar_moeda()

Dogecoin


{'Preço': 'R$0.728',
 'Variação 1h': '0.00%',
 'Variação 24h': '0.54%',
 'Variação 7d': '7.00%',
 'Cap. de Mercado': 'R$105,375,955,266',
 'Volume (24h)': 'R$1,959,632,334',
 'Fornecimento Circulante': '144,737,596,384 DOGE'}